In [11]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Flatten
# from keras.layers import Dense
# from keras.layers import LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np
import pandas as pd
import keras
# config=tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# sess=tf.compat.v1.Session(config=config)
# keras.backend.tensorflow_backend.set_session(sess)


In [12]:
# a = [1,2,3]
# b = np.array(a)
# c = []
# c.append([0,1])
# c.append([1,0])
# print(c)

#数据预处理

In [13]:

# dataset = read_csv('C:/Users/20987/Desktop/dataset/开心果数据集/Pistachio_DeepLearning/Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv')
# #打乱顺序
# dataset =  dataset.reindex(np.random.permutation(dataset.index))
# dataset.to_csv('randomIndex.csv',index=False)


In [14]:
#转成有监督数据
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    #数据序列(也将就是input) input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
        #预测数据（input对应的输出值） forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    #拼接 put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # 删除值为NAN的行 drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [15]:
'''
Date: 2022-08-08 12:39:30
LastEditors: ZSudoku
LastEditTime: 2022-08-12 18:51:52
FilePath: \Pistachio_DeepLearning\LSTM.ipynb
'''

##数据预处理 load dataset
dataset = read_csv('./randomIndex.csv')
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']
dataset = dataset.loc[:,cols]
# print(dataset.columns)

# lisIndex = [i for i in range(len(dataset))]
# dataset['data'] = lisIndex
# dataset.index.name = 'date'
print(dataset.shape)
Class_list = []
for i in dataset.index:
    if(dataset['Class'][i] not in Class_list):
        Class_list.append(dataset['Class'][i])
for i in dataset.index:
    for j in range(len(Class_list)):
        if dataset.loc[i , 'Class'] == Class_list[j]:
            dataset.loc[i , 'Class'] = int(j)
dataset= dataset.astype({'Class' : 'int'})

# 将n-1列的属性字符串列转换为浮点数，第n列为分类的类别
Pistachio_columns = dataset.columns
for i in range(len(dataset.columns)-1):
    dataset= dataset.astype({'%s'%Pistachio_columns[i] : 'float64'})

# dataset = read_csv('C:/Users/20987/Desktop/LSTM/LSTM系列/LSTM多变量3/data_set/pollution.csv', header=0, index_col=0)
values = dataset.values
#标签编码 integer encode direction
# encoder = LabelEncoder()
# values[:, 4] = encoder.fit_transform(values[:, 4])
# #保证为float ensure all data is float
# values = values.astype('float32')
print(values.shape)
#归一化 normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# print(type(scaled))
# print('scaled',scaled)
# reframed = scaled
#转成有监督数据 frame as supervised learning
# reframed = series_to_supervised(scaled)
# reframed = series_to_supervised(scaled, 1, 1)
reframed = pd.DataFrame(scaled)
print(reframed.columns)
reframed[0] = reframed[0].values.astype(int)
#删除不预测的列 drop columns we don't want to predict
# reframed.drop(reframed.columns[[i for i in range(30,58)]], axis=1, inplace=True)
print(reframed.head())
reframed.to_csv('reframed.csv')


(2148, 29)
(2148, 29)
RangeIndex(start=0, stop=29, step=1)
   0         1         2         3         4         5         6         7   \
0   0  0.642081  0.160647  0.593442  0.496208  0.718885  0.712189  0.983051   
1   0  0.793323  0.187556  0.586551  0.656489  0.558150  0.839426  0.995333   
2   0  0.789416  0.203946  0.692833  0.588882  0.682838  0.836252  0.990911   
3   1  0.334310  0.103649  0.494137  0.227402  0.888234  0.417490  0.865389   
4   1  0.223376  0.654282  0.228366  0.275383  0.755838  0.294430  0.567183   

         8         9   ...        19        20        21        22        23  \
0  0.565161  0.850458  ...  0.834170  0.466812  0.395350  0.304327  0.283540   
1  0.712046  0.836979  ...  0.864009  0.523290  0.486902  0.325652  0.226442   
2  0.710132  0.666836  ...  0.457268  0.430712  0.332943  0.228891  0.271398   
3  0.288334  0.694049  ...  0.512903  0.632895  0.568431  0.382078  0.272320   
4  0.255513  0.547050  ...  0.409394  0.427662  0.366620  0.220203

数据准备

In [16]:
def Stock_Price_LSTM_Data_Precesing(reframed):
    #把数据分为训练数据和测试数据 split into train and test sets
    values = reframed.values
    #拿10%的长度训练
    n_train = int(len(dataset) * 0.1)
    #划分训练数据和测试数据
    train = values[:n_train, :]
    test = values[n_train:, :]
    #拆分输入输出 split into input and outputs
    train_X, train_y = train[:, :0], train[:, 0]
    test_X, test_y = test[:, :0], test[:, 0]
    # test_y,train_y ,转换成2维
    train_Y = []
    for i in range(len(train_y)):
        if(train_y[i] == 0):
            train_Y.append([1,0])
        else:
            train_Y.append([0,1])
    # print(train_Y)

    test_Y = []
    for i in range(len(test_y)):
        if(test_y[i] == 0):
            test_Y.append([1,0])
        else:
            test_Y.append([0,1])
    # print(test_Y)

    # for i in range(len(test_y)):
    train_y = np.array(train_Y)
    test_y = np.array(test_Y)
    #reshape输入为LSTM的输入格式 reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 2, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 2, test_X.shape[1]))
    print ('train_x.shape, train_y.shape, test_x.shape, test_y.shape')
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    return train_X, train_y ,test_X, test_y

In [17]:
Stock_Price_LSTM_Data_Precesing(reframed)

train_x.shape, train_y.shape, test_x.shape, test_y.shape
(214, 2, 0) (214, 2) (1934, 2, 0) (1934, 2)


(array([], shape=(214, 2, 0), dtype=float64),
 array([[1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0

In [18]:
# ##模型定义 design network
# train_X, train_y ,test_X, test_y = Stock_Price_LSTM_Data_Precesing(reframed)
# model = Sequential()
# model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]),activation='relu'))
# model.add(Dense(1))
# model.compile(loss='mae', optimizer='adam')
# #模型训练 fit network
# history = model.fit(train_X, train_y, epochs=25, batch_size=72, validation_data=(test_X, test_y), verbose=2,
#                     shuffle=False)
# #输出 plot history
# pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
# pyplot.legend()
# pyplot.show()


In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
#mymodeltest = my_model((1,28), name='MyModelMLP')
train_X, train_y ,test_X, test_y = Stock_Price_LSTM_Data_Precesing(reframed)
def build_model():
    model = Sequential()
    model.add(LSTM(128,input_shape=(train_X.shape[1], train_X.shape[2]),recurrent_activation='sigmoid',return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128,recurrent_activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
                 optimizer='adam', metrics=['categorical_accuracy'])  
    
    return model
filepath = './models/{val_loss:.2f}_{epoch:02d}_'
checkpoint = ModelCheckpoint(
                    filepath=filepath,
                    save_weights_only=False,
                    monitor='categorical_accuracy',
                    mode='max',
                    save_best_only=True)
model = build_model()
model.fit(train_X,train_y,batch_size=128,epochs=50,callbacks=checkpoint)
# model = KerasClassifier(build_fn=build_model(), epochs=100, batch_size=128)
# kfold = KFold(n_splits=10,shuffle=True,random_state=40)
# result = cross_val_score(model,train_X,train_y,cv=kfold)
# print("============")
# print("mean:",result.mean())
# print("std:",result.std())

train_x.shape, train_y.shape, test_x.shape, test_y.shape
(214, 2, 0) (214, 2) (1934, 2, 0) (1934, 2)
Epoch 1/50


UnknownError:    CUDNN_STATUS_BAD_PARAM
in tensorflow/stream_executor/cuda/cuda_dnn.cc(1140): 'cudnnSetRNNDescriptor_v8( rnn_desc.get(), rnn_algo, rnn_mode, bias_mode, direction_mode, input_mode, data_type, compute_type, math_type, input_size, hidden_size, proj_size, num_layers, dropout_desc.handle(), aux_flags)'
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm_2/PartitionedCall]] [Op:__inference_train_function_11261]

Function call stack:
train_function -> train_function -> train_function


第一层，LSTM(神经元数量，输入，激活函数，return_sequences=True(循环神经网络))
第二层：与第一层相同
第三层，与第二层相同
全连接层
输出层
编译：（优化器，损失函数，评价函数）

In [ ]:
'''
Date: 2022-08-08 12:39:30
LastEditors: ZSudoku
LastEditTime: 2022-08-08 17:16:12
FilePath: \Pistachio_DeepLearning\LSTM.ipynb
'''
lstm_layers = [1,2,3]
dense_layers = [1,2,3]
units = [16,32]

from tensorflow.keras.callbacks import ModelCheckpoint
for the_lstm_layers in lstm_layers:
        for the_dense_layers in dense_layers:
            for the_units in units:
                filepath = './models/{val_loss:.2f}_{epoch:02d}_'+f'_lstm_{the_lstm_layers}_dense_{the_dense_layers}_unit_{the_units}'
                checkpoint = ModelCheckpoint(
                    filepath=filepath,
                    save_weights_only=False,
                    monitor='val_loss',
                    mode='min',
                    save_best_only=True)
                train_X, train_y ,test_X, test_y = Stock_Price_LSTM_Data_Precesing(reframed)
                model = Sequential()
                #第一层
                # print(train_X.shape[0])
                # print(train_X.shape[1])
                # print(train_X.shape[2])
                model.add(LSTM(the_units,input_shape=(train_X.shape[1], train_X.shape[2]),activation='relu',return_sequences=True))
                model.add(Dropout(0.8))
                #第二层
                for i in range(the_lstm_layers):
                    model.add(LSTM(the_units,activation='relu',return_sequences=True))
                    model.add(Dropout(0.8))

                model.add(LSTM(the_units,activation='relu'))
                model.add(Dropout(0.8))
                # 全连接层
                for i in range(the_dense_layers):
                    model.add(Dense(the_units,activation='relu'))
                    model.add(Dropout(0.8))
                    
                model.add(Flatten()) 
                #输出层
                model.add(Dense(2,activation='softmax'))
                
                # model.add(Dense(1,activation='softmax'))

                model.compile(loss="categorical_crossentropy", # categorical_crossentropy # sparse_categorical_crossentropy
                            optimizer='adam',
                            metrics=["accuracy"])
                
                # model.compile(optimizer='adam',
                #                 loss='mse',
                #                 metrics=['mape'])

                model.fit(train_X,train_y,batch_size=32,epochs=50,validation_data=(test_X,test_y),callbacks=[checkpoint])

In [ ]:
# #进行预测 make a prediction
# yhat = model.predict(test_X)
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# #预测数据逆缩放 invert scaling for forecast  
# inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
# inv_yhat = scaler.inverse_transform(inv_yhat)
# inv_yhat = inv_yhat[:, 0]
# inv_yhat = np.array(inv_yhat)
# #真实数据逆缩放 invert scaling for actual
# test_y = test_y.reshape((len(test_y), 1))
# inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
# inv_y = inv_y[:, 0]

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/0.68_17__lstm_1_dense_1_unit_16')

In [ ]:
#进行预测 make a prediction
train_X, train_y ,test_X, test_y = Stock_Price_LSTM_Data_Precesing(reframed)
# print ('train_x.shape, train_y.shape, test_x.shape, test_y.shape')
# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
yhat = best_model.predict(test_X)
# print(yhat.shape)
print(yhat)
print(test_y)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
#预测数据逆缩放 invert scaling for forecast  
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:, 0]
inv_yhat = np.array(inv_yhat)
#真实数据逆缩放 invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, 0]

In [ ]:
#画出真实数据和预测数据
# pyplot.plot(inv_yhat,c = 'r',label='prediction')
# pyplot.plot(inv_y,'--b',label='true')
# pyplot.legend()
# pyplot.show()

colors1 = '#C0504D'
colors2 = '#00EEEE'

area1 = np.pi * 2 ** 2    # 半径为2的圆的面积
area2 = np.pi * 0.1 ** 0.1

lisX = [i for i in range(0,len(inv_y))]
pyplot.scatter(lisX,inv_yhat,marker='^',s = area1, c = colors1,label = 'prediction')
pyplot.scatter(lisX,inv_y,marker='o',s = area2, c = colors2,label = 'true')
pyplot.legend()
pyplot.show()


In [ ]:
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)